<a href="https://colab.research.google.com/github/pedroeli07/temperatura_media_insights/blob/Filtragens-iniciais/temperatura_media_insights(primeiras_filtragens).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [25]:
import pandas as pd
# Importe a biblioteca google.colab.drive
import google.colab.drive

# Monte o Google Drive
google.colab.drive.mount('/content/drive')

path = '/content/drive/My Drive/GlobalLandTemperaturesByCity.csv'
df = pd.read_csv(path)
display(df)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,dt,AverageTemperature,AverageTemperatureUncertainty,City,Country,Latitude,Longitude
0,1743-11-01,6.068,1.737,Århus,Denmark,57.05N,10.33E
1,1743-12-01,NaN,NaN,Århus,Denmark,57.05N,10.33E
2,1744-01-01,NaN,NaN,Århus,Denmark,57.05N,10.33E
3,1744-02-01,NaN,NaN,Århus,Denmark,57.05N,10.33E
4,1744-03-01,NaN,NaN,Århus,Denmark,57.05N,10.33E
...,...,...,...,...,...,...,...
8599207,2013-05-01,11.464,0.236,Zwolle,Netherlands,52.24N,5.26E
8599208,2013-06-01,15.043,0.261,Zwolle,Netherlands,52.24N,5.26E
8599209,2013-07-01,18.775,0.193,Zwolle,Netherlands,52.24N,5.26E
8599210,2013-08-01,18.025,0.298,Zwolle,Netherlands,52.24N,5.26E


In [26]:
# Converter Latitude e Longitude para Números
df['Latitude'] = pd.to_numeric(df['Latitude'].astype(str).str.extract('(\d+\.\d+)')[0])
df['Longitude'] = pd.to_numeric(df['Longitude'].astype(str).str.extract('(\d+\.\d+)')[0])
# Renomear colunas
df.rename(columns={
    'AverageTemperature': 'Temperatura Média',
    'AverageTemperatureUncertainty': 'Incerteza Temperatura Média',
    'City': 'Cidade',
    'Country': 'País',
    'Latitude': 'Latitude',
    'Longitude': 'Longitude'
}, inplace=True)

# Supondo que sua coluna 'dt' seja do tipo datetime, se não for, converta-a primeiro
df['dt'] = pd.to_datetime(df['dt'])
# Extrair o ano da coluna 'dt'
df['Ano'] = df['dt'].dt.year
# Agregar a temperatura média para cada cidade em cada ano
df_agg = df.groupby(['Cidade', 'País', 'Latitude', 'Longitude', 'Ano']).agg({
    'Temperatura Média': 'mean',
    'Incerteza Temperatura Média': 'mean'
}).reset_index()

df_agg = df_agg.query('Ano >= 1900')
display(df_agg)

,Cidade,País,Latitude,Longitude,Ano,Temperatura Média,Incerteza Temperatura Média
157,A Coruña,Spain,42.59,8.73,1900,13.267917,0.772417
158,A Coruña,Spain,42.59,8.73,1901,12.773417,0.848167
159,A Coruña,Spain,42.59,8.73,1902,12.828333,0.656500
160,A Coruña,Spain,42.59,8.73,1903,13.028167,0.694583
161,A Coruña,Spain,42.59,8.73,1904,13.349083,0.675500
...,...,...,...,...,...,...,...
718326,Ürümqi,China,44.20,87.20,2009,7.287417,0.520333
718327,Ürümqi,China,44.20,87.20,2010,6.650083,1.028417
718328,Ürümqi,China,44.20,87.20,2011,6.806083,0.702750
718329,Ürümqi,China,44.20,87.20,2012,6.600167,0.842667


In [27]:
# Faça uma cópia do DataFrame
df_copy = df_agg.copy()
# Calcular a média da incerteza média para cada cidade
df_copy['Média Incerteza Média'] = df_copy.groupby('Cidade')['Incerteza Temperatura Média'].transform('mean')
# Excluir a coluna de incerteza média original
df_copy = df_copy.drop('Incerteza Temperatura Média', axis=1)
# Selecione as temperaturas médias dos últimos 5 anos e faça a média
ultimos_5_anos = df_copy[df_copy['Ano'] >= 2009].groupby('Cidade')['Temperatura Média'].mean()
# Selecione as temperaturas médias dos primeiros 5 anos e faça a média
primeiros_5_anos = df_copy[df_copy['Ano'] <= 1905].groupby('Cidade')['Temperatura Média'].mean()
# Calcule a variação de temperatura média
df_copy = df_copy.groupby('Cidade').head(1)  # Seleciona apenas uma linha por cidade
df_copy.loc[:, 'Variação de Temperatura Média'] = ultimos_5_anos.values - primeiros_5_anos.values
# Exclua as colunas Ano e Temperatura Média
df_copy = df_copy.drop(['Ano', 'Temperatura Média'], axis=1)
# Elimine as linhas duplicadas
df_copy = df_copy.drop_duplicates()
# Ordene o DataFrame pela coluna 'Variação de Temperatura Média'
df_copy = df_copy.sort_values(by='Variação de Temperatura Média', ascending=False)
# Exiba o DataFrame resultante
display(df_copy)


,Cidade,País,Latitude,Longitude,Média Incerteza Média,Variação de Temperatura Média
441886,Norilsk,Russia,69.92,88.83,1.110143,2.674072
513614,Quebec,Canada,47.42,72.24,0.371247,2.618036
462529,Ottawa,Canada,45.81,75.00,0.321270,2.605047
407955,Montreal,Canada,45.81,72.69,0.308446,2.561578
576178,Sherbrooke,Canada,45.81,72.69,0.308446,2.561578
...,...,...,...,...,...,...
598018,Sucre,Bolivia,18.48,65.92,0.701475,0.434228
459828,Oruro,Bolivia,18.48,67.61,0.678083,0.425764
499429,Potosí,Bolivia,20.09,66.54,0.737829,0.416425
546596,Salta,Argentina,24.92,65.62,0.539450,0.384742


In [28]:
# Salve o arquivo no Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Salve o arquivo no Google Drive
df_copy.to_csv('/content/drive/My Drive/df_FILTRADO.csv', index=False)
print("Arquivo salvo no seu Google drive com sucesso !!")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Arquivo salvo no seu Google drive com sucesso !!
